In [ ]:
%pip install ultralytics

In [16]:

from ultralytics import YOLO
import cv2

# Load a pre-trained model
model = YOLO('yolov8l.pt')  # n for nano, s for small, m for medium, l for large, x for extra large

output_dir = 'cropped_objects'

In [17]:
image_path = 'example-2.jpg'
image = cv2.imread(image_path)
results = model(image)


0: 384x640 3 persons, 1 book, 112.3ms
Speed: 1.0ms preprocess, 112.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


In [19]:

# Process each detected object
largest_area = 0
largest_object = None
largest_class_id = None
largest_conf = 0

for i, result in enumerate(results):
    boxes = result.boxes.cpu().numpy()
    for j, box in enumerate(boxes):
        r = box.xyxy[0].astype(int)
        class_id = box.cls[0].astype(int)
        conf = box.conf[0]
        
        # Calculate area of the bounding box
        area = (r[2] - r[0]) * (r[3] - r[1])
        
        # Check if this object is larger than the previous largest
        if area > largest_area and model.names[class_id] == 'book':
            largest_area = area
            largest_object = image[r[1]:r[3], r[0]:r[2]]
            largest_class_id = class_id
            largest_conf = conf

# Save only the largest object
if largest_object is not None:
    filename = f"{output_dir}/{model.names[largest_class_id]}_largest_{largest_conf:.2f}.jpg"
    cv2.imwrite(filename, largest_object)
    print(f"Largest object saved as '{filename}'")
else:
    print("No objects detected in the image.")

Largest object saved as 'cropped_objects/book_largest_0.79.jpg'
